[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/langchain/handbook/03-langchain-conversational-memory.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/generation/langchain/handbook/03-langchain-conversational-memory.ipynb)

#### [LangChain Handbook](https://pinecone.io/learn/langchain)

# Conversational Memory

Conversational memory is how chatbots can respond to our queries in a chat-like manner. It enables a coherent conversation, and without it, every query would be treated as an entirely independent input without considering past interactions.

The memory allows a _"agent"_ to remember previous interactions with the user. By default, agents are *stateless* — meaning each incoming query is processed independently of other interactions. The only thing that exists for a stateless agent is the current input, nothing else.

There are many applications where remembering previous interactions is very important, such as chatbots. Conversational memory allows us to do that.

In this notebook we'll explore this form of memory in the context of the LangChain library.

We'll start by importing all of the libraries that we'll be using in this example.

In [1]:
!pip install -qU langchain openai tiktoken

In [2]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory, 
                                                  ConversationSummaryMemory, 
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted. 

In [4]:
OPENAI_API_KEY = getpass()

··········


In [5]:
llm = OpenAI(
    temperature=0, 
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo'  # can be used with llms like 'text-davinci-003'
)

Later we will make use of a `count_tokens` utility function. This will allow us to count the number of tokens we are using for each call. We define it as so:

In [6]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

Now let's dive into **Conversational Memory**.

## What is memory?

**Definition**: Memory is an agent's capacity of remembering previous interactions with the user (think chatbots)

The official definition of memory is the following:


> By default, Chains and Agents are stateless, meaning that they treat each incoming query independently. In some applications (chatbots being a GREAT example) it is highly important to remember previous interactions, both at a short term but also at a long term level. The concept of “Memory” exists to do exactly that.


As we will see, although this sounds really straightforward there are several different ways to implement this memory capability.

Before we delve into the different memory modules that the library offers, we will introduce the chain we will be using for these examples: the `ConversationChain`.

As always, when understanding a chain it is interesting to peek into its prompt first and then take a look at its `._call` method. As we saw in the chapter on chains, we can check out the prompt by accessing the `template` within the `prompt` attribute.

In [7]:
conversation = ConversationChain(
    llm=llm, 
)

In [8]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


Interesting! So this chain's prompt is telling it to chat with the user and try to give truthful answers. If we look closely, there is a new component in the prompt that we didn't see when we were tinkering with the `LLMMathChain`: _history_. This is where our memory will come into play.

What is this chain doing with this prompt? Let's take a look.

In [9]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        known_values = self.prep_inputs(inputs.copy())
        return self.apply([known_values])[0]
     def apply(self, input_list: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        response = self.generate(input_list)
        return self.create_outputs(response)



Nothing really magical going on here, just a straightforward pass through an LLM. In fact, this chain inherits these methods directly from the `LLMChain` without any modification:

In [10]:
print(inspect.getsource(LLMChain._call), inspect.getsource(LLMChain.apply))

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        known_values = self.prep_inputs(inputs.copy())
        return self.apply([known_values])[0]
     def apply(self, input_list: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        response = self.generate(input_list)
        return self.create_outputs(response)



So basically this chain combines an input from the user with the conversation history to generate a meaningful (and hopefully truthful) response.

Now that we've understood the basics of the chain we'll be using, we can get into memory. Let's dive in!

## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.

### Memory type #1: ConversationBufferMemory

The `ConversationBufferMemory` does just what its name suggests: it keeps a buffer of the previous conversation excerpts as part of the context in the prompt.

**Key feature:** _the conversation buffer memory keeps the previous pieces of conversation completely unmodified, in their raw form._

In [11]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

We pass a user prompt the the `ConversationBufferMemory` like so:

In [12]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning! How can I assist you today?'}

This one call used a total of `85` tokens, but we can't see that from the above. If we'd like to count the number of tokens being used we just pass our conversation chain object and the message we'd like to input via the `count_tokens` function we defined earlier:

In [13]:
count_tokens(
    conversation_buf, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 169 tokens


"That's a fascinating topic! Large Language Models, such as GPT-3, have shown great promise in generating human-like text. However, integrating external knowledge can enhance their capabilities even further. There are several approaches to achieving this, such as using knowledge graphs or incorporating structured data. Would you like me to provide more details on these approaches?"

In [14]:
count_tokens(
    conversation_buf,
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 277 tokens


"Sure! One approach is to use pre-existing knowledge graphs, such as Wikidata or DBpedia, to provide additional context to the language model. Another approach is to incorporate structured data, such as tables or databases, into the model's training data. Additionally, some researchers are exploring the use of reinforcement learning to teach the model how to interact with external knowledge sources. These are just a few examples, but there are many other possibilities to explore."

In [15]:
count_tokens(
    conversation_buf, 
    "Which data source types could be used to give context to the model?"
)

Spent a total of 399 tokens


'There are many different types of data sources that could be used to give context to the model. Some examples include structured data sources like databases or tables, unstructured data sources like text documents or web pages, and knowledge graphs like Wikidata or DBpedia. Additionally, some researchers are exploring the use of social media data or user-generated content to provide additional context to the model. Ultimately, the choice of data source will depend on the specific use case and the type of information that is most relevant to the task at hand.'

In [16]:
count_tokens(
    conversation_buf, 
    "What is my aim again?"
)

Spent a total of 446 tokens


'Your aim is to explore the potential of integrating Large Language Models with external knowledge. You want to analyze the different possibilities and data sources that could be used to give context to the model.'

Our LLM with `ConversationBufferMemory` can clearly remember earlier interactions in the conversation. Let's take a closer look to how the LLM is saving our previous conversation. We can do this by accessing the `.buffer` attribute for the `.memory` in our chain.

In [17]:
print(conversation_buf.memory.buffer)


Human: Good morning AI!
AI: Good morning! How can I assist you today?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI: That's a fascinating topic! Large Language Models, such as GPT-3, have shown great promise in generating human-like text. However, integrating external knowledge can enhance their capabilities even further. There are several approaches to achieving this, such as using knowledge graphs or incorporating structured data. Would you like me to provide more details on these approaches?
Human: I just want to analyze the different possibilities. What can you think of?
AI: Sure! One approach is to use pre-existing knowledge graphs, such as Wikidata or DBpedia, to provide additional context to the language model. Another approach is to incorporate structured data, such as tables or databases, into the model's training data. Additionally, some researchers are exploring the use of reinforcement learning to teach 

Nice! So every piece of our conversation has been explicitly recorded and sent to the LLM in the prompt.

### Memory type #2: ConversationSummaryMemory

The problem with the `ConversationBufferMemory` is that as the conversation progresses, the token count of our context history adds up. This is problematic because we might max out our LLM with a prompt that is too large to be processed.

Enter `ConversationSummaryMemory`.

Again, we can infer from the name what is going on.. we will keep a summary of our previous conversation snippets as our history. How will we summarize these? LLM to the rescue.

**Key feature:** _the conversation summary memory keeps the previous pieces of conversation in a summarized form, where the summarization is performed by an LLM._

In this case we need to send the llm to our memory constructor to power its summarization ability.

In [18]:
conversation_sum = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=llm)
)

When we have an llm, we always have a prompt ;) Let's see what's going on inside our conversation summary memory:

In [19]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


Cool! So each new interaction is summarized and appended to a running summary as the memory of our chain. Let's see how this works in practice!

In [20]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum, 
    "Good morning AI!"
)

Spent a total of 252 tokens


'Good morning! How can I assist you today?'

In [21]:
count_tokens(
    conversation_sum, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 602 tokens


"That's a fascinating topic! Large Language Models, such as GPT-3, have shown great promise in generating human-like text. When it comes to integrating external knowledge, there are a few approaches that can be taken. One is to use pre-existing knowledge graphs, such as Wikidata or DBpedia, to augment the model's understanding of the world. Another approach is to fine-tune the model on a specific domain, such as medicine or law, to give it a deeper understanding of the relevant concepts and terminology. Would you like me to provide more information on these approaches, or do you have any specific questions in mind?"

In [22]:
count_tokens(
    conversation_sum, 
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 714 tokens


"Well, there are a few different approaches you could take when it comes to integrating external knowledge with Large Language Models. One option is to use pre-existing knowledge graphs, which can provide a structured representation of knowledge that can be easily integrated with the model. Another approach is to fine-tune the model on a specific domain, which can help it better understand the specific concepts and terminology related to that domain. Of course, there are pros and cons to each approach, and the best option will depend on your specific use case. If you have any specific questions or would like more information on either approach, I'd be happy to provide it."

In [23]:
count_tokens(
    conversation_sum, 
    "Which data source types could be used to give context to the model?"
)

Spent a total of 741 tokens


'There are many different types of data sources that could be used to give context to a Large Language Model. Some examples include structured data sources like knowledge graphs or databases, unstructured data sources like text corpora or social media feeds, and even sensor data or other types of real-time data streams. The choice of data source will depend on the specific use case and the type of information that is most relevant to the task at hand. Would you like me to provide more information on any of these options?'

In [24]:
count_tokens(
    conversation_sum, 
    "What is my aim again?"
)

Spent a total of 689 tokens


"I'm sorry, I don't have that information. Could you please clarify your question or provide more context?"

In [25]:
print(conversation_sum.memory.buffer)

The human expresses interest in integrating Large Language Models with external knowledge, and the AI suggests two approaches: using pre-existing knowledge graphs or fine-tuning the model on a specific domain. The AI offers to provide more information or answer any specific questions the human may have. The human asks for more possibilities, and the AI explains that there are many different types of data sources that could be used to give context to a Large Language Model. The AI emphasizes that the choice of data source will depend on the specific use case and the type of information that is most relevant to the task at hand. The human then asks for clarification on their aim, to which the AI apologizes for not having that information and asks for more context.


You might be wondering.. if the aggregate token count is greater in each call here than in the buffer example, why should we use this type of memory? Well, if we check out buffer we will realize that although we are using more tokens in each instance of our conversation, our final history is shorter. This will enable us to have many more interactions before we reach our prompt's max length, making our chatbot more robust to longer conversations.

We can count the number of tokens being used (without making a call to OpenAI) using the `tiktoken` tokenizer like so:

In [26]:
# initialize tokenizer
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

# show number of tokens for the memory used by each memory type
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}'
)

Buffer memory conversation length: 386
Summary memory conversation length: 144


_Practical Note: the `text-davinci-003` model [has](https://platform.openai.com/docs/api-reference/completions/create#completions/create-max_tokens) a large max tokens count of 4096 tokens between prompt and answer._

### Memory type #3: ConversationBufferWindowMemory

Another great option for these cases is the `ConversationBufferWindowMemory` where we will be keeping a few of the last interactions in our memory but we will intentionally drop the oldest ones - short-term memory if you'd like. Here the aggregate token count **and** the per-call token count will drop noticeably. We will control this window with the `k` parameter.

**Key feature:** _the conversation buffer window memory keeps the latest pieces of the conversation in raw form_

In [27]:
conversation_bufw = ConversationChain(
    llm=llm, 
    memory=ConversationBufferWindowMemory(k=2)
)

In [28]:
count_tokens(
    conversation_bufw, 
    "Good morning AI!"
)

Spent a total of 79 tokens


'Good morning! How can I assist you today?'

In [29]:
count_tokens(
    conversation_bufw, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 169 tokens


"That's a fascinating topic! Large Language Models, such as GPT-3, have shown great promise in generating human-like text. However, integrating external knowledge can enhance their capabilities even further. There are several approaches to achieving this, such as using knowledge graphs or incorporating structured data. Would you like me to provide more details on these approaches?"

In [30]:
count_tokens(
    conversation_bufw, 
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 277 tokens


"Sure! One approach is to use pre-existing knowledge graphs, such as DBpedia or Wikidata, to provide additional context to the language model. Another approach is to incorporate structured data, such as tables or databases, into the model's training data. Additionally, some researchers are exploring the use of reinforcement learning to teach the model how to interact with external knowledge sources. These are just a few examples, but there are many other possibilities to explore."

In [31]:
count_tokens(
    conversation_bufw, 
    "Which data source types could be used to give context to the model?"
)

Spent a total of 343 tokens


"There are many different types of data sources that could be used to give context to the model. Some examples include knowledge graphs, databases, tables, text corpora, and even social media data. The choice of data source will depend on the specific use case and the type of information that is needed to enhance the model's capabilities."

In [32]:
count_tokens(
    conversation_bufw, 
    "What is my aim again?"
)

Spent a total of 292 tokens


"I'm sorry, I don't have enough context to know what your specific aim is. Could you please provide more information or clarify your question?"

As we can see, it effectively 'fogot' what we talked about in the first interaction. Let's see what it 'remembers'. Given that we set k to be `2`, we would expect it remembers only our last two interactions.

We need to access a special method here since, in this memory type, the buffer is first passed through this method to be sent later to the llm.

In [33]:
bufw_history = conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']

In [34]:
print(bufw_history)

Human: Which data source types could be used to give context to the model?
AI: There are many different types of data sources that could be used to give context to the model. Some examples include knowledge graphs, databases, tables, text corpora, and even social media data. The choice of data source will depend on the specific use case and the type of information that is needed to enhance the model's capabilities.
Human: What is my aim again?
AI: I'm sorry, I don't have enough context to know what your specific aim is. Could you please provide more information or clarify your question?


Makes sense. 

On the plus side, we are shortening our conversation length when compared to buffer memory _without_ a window:

In [35]:
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}\n'
    f'Buffer window memory conversation length: {len(tokenizer.encode(bufw_history))}'
)

Buffer memory conversation length: 386
Summary memory conversation length: 144
Buffer window memory conversation length: 123


_Practical Note: We are using `k=2` here for illustrative purposes, in most real world applications you would need a higher value for k._

### More memory types!

Given that we understand memory already, we will present a few more memory types here and hopefully a brief description will be enough to understand their underlying functionality.

#### ConversationSummaryBufferMemory

**Key feature:** _the conversation summary memory keeps a summary of the earliest pieces of conversation while retaining a raw recollection of the latest interactions._

#### ConversationKnowledgeGraphMemory

This is a super cool memory type that was introduced just [recently](https://twitter.com/LangChainAI/status/1625158388824043522). It is based on the concept of a _knowledge graph_ which recognizes different entities and connects them in pairs with a predicate resulting in (subject, predicate, object) triplets. This enables us to compress a lot of information into highly significant snippets that can be fed into the model as context. If you want to understand this memory type in more depth you can check out [this](https://apex974.com/articles/explore-langchain-support-for-knowledge-graph) blogpost.

**Key feature:** _the conversation knowledge graph memory keeps a knowledge graph of all the entities that have been mentioned in the interactions together with their semantic relationships._

In [36]:
# you may need to install this library
# !pip install -qU networkx

In [37]:
conversation_kg = ConversationChain(
    llm=llm, 
    memory=ConversationKGMemory(llm=llm)
)

In [38]:
count_tokens(
    conversation_kg, 
    "My name is human and I like mangoes!"
)

Spent a total of 1480 tokens


"Hello, Human! It's nice to meet you. Mangoes are a delicious fruit. Did you know that they are native to South Asia and have been cultivated for over 4,000 years? They are also a great source of vitamin C and fiber. Do you have a favorite way to eat mangoes?"

The memory keeps a knowledge graph of everything it learned so far.

In [39]:
conversation_kg.memory.kg.get_triples()

[('Human', 'human', 'has a name'), ('Human', 'mangoes', 'likes')]

#### ConversationEntityMemory

**Key feature:** _the conversation entity memory keeps a recollection of the main entities that have been mentioned, together with their specific attributes._

The way this works is quite similar to the `ConversationKnowledgeGraphMemory`, you can refer to the [docs](https://langchain.readthedocs.io/en/latest/modules/memory/examples/entity_summary_memory.html) if you want to see it in action. 

## What else can we do with memory?

There are several cool things we can do with memory in langchain. We can:
* implement our own custom memory module
* use multiple memory modules in the same chain
* combine agents with memory and other tools

If this piques your interest, we suggest you to go take a look at the memory [how-to](https://langchain.readthedocs.io/en/latest/modules/memory/how_to_guides.html) section in the docs!